In [27]:
%matplotlib inline


Autograd: automatic differentiation
===================================

Central to all neural networks in PyTorch is the ``autograd`` package.
Let’s first briefly visit this, and we will then go to training our
first neural network.


The ``autograd`` package provides automatic differentiation for all operations
on Tensors. It is a define-by-run framework, which means that your backprop is
defined by how your code is run, and that every single iteration can be
different.

Let us see this in more simple terms with some examples.

Variable
--------

``autograd.Variable`` is the central class of the package. It wraps a
Tensor, and supports nearly all of operations defined on it. Once you
finish your computation you can call ``.backward()`` and have all the
gradients computed automatically.

You can access the raw tensor through the ``.data`` attribute, while the
gradient w.r.t. this variable is accumulated into ``.grad``.

.. figure:: /_static/img/Variable.png
   :alt: Variable

   Variable

There’s one more class which is very important for autograd
implementation - a ``Function``.

``Variable`` and ``Function`` are interconnected and build up an acyclic
graph, that encodes a complete history of computation. Each variable has
a ``.grad_fn`` attribute that references a ``Function`` that has created
the ``Variable`` (except for Variables created by the user - their
``grad_fn is None``).

If you want to compute the derivatives, you can call ``.backward()`` on
a ``Variable``. If ``Variable`` is a scalar (i.e. it holds a one element
data), you don’t need to specify any arguments to ``backward()``,
however if it has more elements, you need to specify a ``grad_output``
argument that is a tensor of matching shape.



In [28]:
import torch
from torch.autograd import Variable

Create a variable:



In [29]:
x = Variable(torch.ones(2, 2), requires_grad=True)
print(x)

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]



Do an operation of variable:



In [30]:
y = x + 2
print(y)

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]



``y`` was created as a result of an operation, so it has a ``grad_fn``.



In [31]:
print(y.grad_fn)

Do more operations on y



In [32]:
z = y * y * 5
out = z.mean()

print(z, out)

Variable containing:
 45  45
 45  45
[torch.FloatTensor of size 2x2]
 Variable containing:
 45
[torch.FloatTensor of size 1]



Gradients
---------
let's backprop now
``out.backward()`` is equivalent to doing ``out.backward(torch.Tensor([1.0]))``



In [33]:
out.backward()

print gradients d(out)/dx




In [34]:
print(x.grad)

Variable containing:
 7.5000  7.5000
 7.5000  7.5000
[torch.FloatTensor of size 2x2]



You should have got a matrix of ``4.5``. Let’s call the ``out``
*Variable* “$o$”.
We have that $o = \frac{1}{4}\sum_i z_i$,
$z_i = 3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$.
Therefore,
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$, hence
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.



You can do many crazy things with autograd!



In [35]:
x = torch.randn(3)
print(x)


 0.1434
 2.4304
 0.4210
[torch.FloatTensor of size 3]



In [36]:
x = Variable(x, requires_grad=True)

y = x * 2
print( x,y)

Variable containing:
 0.1434
 2.4304
 0.4210
[torch.FloatTensor of size 3]
 Variable containing:
 0.2869
 4.8608
 0.8421
[torch.FloatTensor of size 3]



In [49]:
y = y * 2
print(y)
print(y.data.norm())

Variable containing:
  4700.1108
 79639.4141
 13796.4980
[torch.FloatTensor of size 3]

80962.1558043174


In [50]:
while y.data.norm() < 1000:
    print(y.data.norm())
    y = y * 2

print(y)

Variable containing:
  4700.1108
 79639.4141
 13796.4980
[torch.FloatTensor of size 3]



In [51]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

Variable containing:
  3276.8000
 32768.0000
     3.2768
[torch.FloatTensor of size 3]



**Read Later:**

Documentation of ``Variable`` and ``Function`` is at
http://pytorch.org/docs/autograd

